# **Use YOLO-cls !**

## 0.미션


### (1) 미션1
여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 **문제에 적합한** UltraLytics YOLO-cls 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 바꿔야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - UltraLytics YOLO-cls 모델에서 요구하는 데이터셋 폴더의 구조가 있습니다.
    - [UltraLytics YOLO-cls 모델의 데이터셋 구조 링크](https://docs.ultralytics.com/datasets/classify/)

### (2) 미션2
데이터셋의 폴더 구조를 **학습에 적합한 형태**로 만들었다면, **사전 학습된 UltraLytics YOLO-cls 모델**에 Transfer Learning을 수행합니다.

- 1) UltraLytics YOLO-cls 모델 선택
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)
- 2) 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)
- 3) 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/#inference-arguments)
- 4) 해당 UltraLytics YOLO-cls 모델을 **반드시** 저장합니다.
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 구글콜랩
        * 구글 드라이브 바로 밑에 project4 폴더를 만드세요.
        * 데이터 파일을 복사해 넣습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

* 구글 드라이브 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/project4'

### (2) 라이브러리 설치 및 불러오기

* 라이브러리 로딩

In [4]:
## colab에서 세션 재시작을 요구하는 팝업이 뜨면 재시작 누르세요.
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 48.7 MB/s eta 0:00:00


## 2.미션1

여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 **문제에 적합한** UltraLytics YOLO-cls 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 바꿔야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - UltraLytics YOLO-cls 모델에서 요구하는 데이터셋 폴더의 구조가 있습니다.
    - [UltraLytics YOLO-cls 모델의 데이터셋 구조 링크](https://docs.ultralytics.com/datasets/classify/)

### (1) 데이터셋 불러오기

* **세부 요구사항**
    - 데이터셋을 불러옵니다.
        - 데이터셋은 두 개의 압축 파일이어야 합니다.
            1. lfw-deepfunneled.zip : Labeled Faces in the Wild 데이터셋
                - 압축 파일을 로컬에 다운로드 받아서 **어떤 구조**인지 확인하세요.
            2. 여러분의 얼굴 이미지 데이터셋
                - 여러분의 얼굴 이미지가 담긴 **압축 파일**을 **Google Drive에 업로드** 하기를 권장합니다.
                    - 이미지 파일 하나하나 업로드 하면 시간이 오래 걸립니다.
    - 데이터셋 압축 파일을 **Colab에 폴더를 생성한 후 해제**하세요.
        - 데이터셋 폴더를 **본인 얼굴 폴더, LFW 폴더로 나누어** 생성하는 것을 권장합니다.
        - 만일 두 압축 파일을 하나의 폴더에 모두 해제하면 전처리가 더 까다로워질 것입니다.
    - 예시 코드에서 사용한 라이브러리
        - os, zipfile

In [5]:
import keras
import zipfile
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import numpy as np
from keras.utils import load_img, img_to_array

import numpy as np
import matplotlib.pyplot as plt

#### 1) 본인 얼굴 이미지 데이터셋 불러오기

In [6]:
myimg =  '/content/drive/MyDrive/project4/my_face_exam.zip'
myimg
## Colab에 생성할 본인 얼굴 폴더 경로
extract_folder = '/content/my_face'

## 위의 경로에 폴더가 없을 때 생성
if not os.path.exists(extract_folder) :
    os.makedirs(extract_folder)

## 위의 경로에 압축을 해제
with zipfile.ZipFile(myimg, 'r') as zip_ref :
    file_list = zip_ref.namelist()

    for f in file_list :
        if not f.endswith('/') and f.lower().endswith('.jpg') :
            file_name = os.path.basename(f)

            if not file_name.startswith('._') :
                d_path = os.path.join(extract_folder, file_name)

                with zip_ref.open(f) as source, open(d_path, 'wb') as target :
                    target.write(source.read())

#### 2) 다른 얼굴 이미지 데이터셋 불러오기

In [7]:
faceimg =  '/content/drive/MyDrive/project4/Datasets/Keras/lfw-deepfunneled.zip'
faceimg
## Colab에 생성할 다른사람 얼굴 폴더 경로
extract_folder = '/content/other_face'

## 위의 경로에 폴더가 없을 때 생성
if not os.path.exists(extract_folder) :
    os.makedirs(extract_folder)

## 위의 경로에 압축을 해제
with zipfile.ZipFile(faceimg, 'r') as zip_ref :
    file_list = zip_ref.namelist()

    for f in file_list :
        if not f.endswith('/') and f.lower().endswith('.jpg') :
            file_name = os.path.basename(f)

            if not file_name.startswith('._') :
                d_path = os.path.join(extract_folder, file_name)

                with zip_ref.open(f) as source, open(d_path, 'wb') as target :
                    target.write(source.read())

In [8]:
## 생성된 다른 사람 얼굴 이미지 데이터 폴더 안의 이미지 수
len(os.listdir(extract_folder) )

13233

### (2) 데이터셋 전처리

* **세부 요구사항**
    - 데이터셋을 전처리 합니다.
        - YOLO-cls 모델이 요구하는 폴더 구조를 만듭니다.
            1. Datasets라는 폴더를 생성합니다.
            2. Training set, Validation set, Test set(선택 사항) 각 데이터셋이 들어갈 폴더를 생성합니다.
            3. 각 데이터셋 폴더에 분류할 클래스의 이름을 가진 폴더를 생성합니다.
        - 폴더 구조에 맞게 데이터를 분배합니다.
    - 예시 코드에서 사용한 라이브러리
        - os, glob, random, shutil, numpy

#### 1) 모델이 요하는 구조의 폴더 생성

In [ ]:
# 이미지 삭제 코드
import os

# 삭제할 폴더 경로
folders = ['/content/my_face', '/content/other_face']

# 각 폴더 내 파일 삭제
for folder in folders:
    if os.path.exists(folder):
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            if os.path.isfile(file_path):  # 파일인지 확인
                os.remove(file_path)  # 파일 삭제
        print(f"{folder} 폴더의 모든 파일이 삭제되었습니다.")
    else:
        print(f"{folder} 폴더가 존재하지 않습니다.")


In [ ]:
#### 이미지 파일 삭제 코드
import os

# 삭제할 폴더 경로
base_dir = "/content/Datasets"
datasets = ['train', 'val','test']
classes = ['my_face', 'other_face']

# 각 폴더 내 파일 삭제
for dataset in datasets:
    for class_name in classes:
        folder_path = os.path.join(base_dir, dataset, class_name)

        if os.path.exists(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if os.path.isfile(file_path):  # 파일인지 확인
                    os.remove(file_path)  # 파일 삭제
            print(f"{folder_path} 폴더의 모든 파일이 삭제되었습니다.")
        else:
            print(f"{folder_path} 폴더가 없습니다.")


In [13]:
import os

# 상위 폴더 생성
base_dir = '/content/Datasets'
os.makedirs(base_dir, exist_ok=True)

# 데이터셋 및 클래스 이름 정의
datasets = ['train', 'val', 'test']
classes = ['my_face', 'other_face']

# 폴더 구조 생성
for dataset in datasets:
    dataset_dir = os.path.join(base_dir, dataset)
    os.makedirs(dataset_dir, exist_ok=True)

    for class_name in classes:
        class_dir = os.path.join(dataset_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

print("폴더 구조가 성공적으로 생성되었습니다.")


폴더 구조가 성공적으로 생성되었습니다.


#### 2) 각 폴더에 이미지 데이터 옮기기

In [14]:
import glob
import random
import shutil
import numpy as np

from keras.utils import load_img, img_to_array
from keras.utils import image_dataset_from_directory

In [15]:
## 본인 얼굴 데이터가 있는 폴더 경로의 파일 전체를 정렬하여 리스트화
img_list_my = sorted(glob.glob('/content/my_face/*',))

## 다른 얼굴 데이터가 있는 폴더 경로의 파일 전체를 정렬하여 리스트화
img_list_other = sorted(glob.glob('/content/other_face/*'))

## 이미지 갯수 확인
len(img_list_my), len(img_list_other)

(3000, 13233)

In [16]:
## 얼굴 데이터를 Training set, Test set으로 분할하기 위한 사전 작업
## 분할 재현성을 위한 난수 고정
random.seed(2024)
random.shuffle(img_list_my)
random.shuffle(img_list_other)

img_list_my[:5], img_list_other[:5]

(['/content/my_face/my_face_753.jpg',
  '/content/my_face/my_face_200.jpg',
  '/content/my_face/my_face_761.jpg',
  '/content/my_face/my_face_1128.jpg',
  '/content/my_face/my_face_2092.jpg'],
 ['/content/other_face/Colin_Powell_0233.jpg',
  '/content/other_face/Julia_Glass_0001.jpg',
  '/content/other_face/Willie_Nelson_0001.jpg',
  '/content/other_face/Habib_Rizieq_0004.jpg',
  '/content/other_face/Hugo_Chavez_0010.jpg'])

In [17]:
## Test set의 비율 설정
test_size = 0.2

## 생성될 폴더의 경로
tr_data = '/content/Datasets/train'
te_data = '/content/Datasets/val'

## 나의 얼굴 파일 리스트와 다른 얼굴 파일 리스트에 대한 반복문
for i_l in [img_list_my, img_list_other] :
    ## 리스트의 길이 체크
    # list_len = len(i_l)
    list_len = 3000  ## 예시 파일의 이미지 갯수가 맞지 않아서 11000개까지만 사용
    ## 데이터 분할을 위한 인덱스 설정
    split_idx = int(list_len * (1 - test_size) )

    ## 인덱스를 이용해 상위 리스트를 Training set, Test set 2가지로 세분화
    list_tr = i_l[ : split_idx]
    list_te = i_l[split_idx : list_len]

    ## 현재 리스트가 나의 얼굴 파일 리스트와 같다면
    if i_l == img_list_my :
        ## "나의 얼굴 파일 리스트"의 파일을 Training set 폴더 안의 "나의 얼굴 폴더"로 복사
        ## 이동이 잘못 되었을 경우를 생각하여 복사로 진행
        for file_path in list_tr :
            f_name = file_path.split('/')
            shutil.copy(src=file_path,
                        dst=tr_data+'/my_face/'+f_name[-1]
                        )
            print(f'파일 이동 완료 : {f_name[-1]}')

        ## "나의 얼굴 파일 리스트"의 파일을 Test set 폴더 안의 "나의 얼굴 폴더"로 복사
        ## 이동이 잘못 되었을 경우를 생각하여 복사로 진행
        for file_path in list_te :
            f_name = file_path.split('/')
            shutil.copy(src=file_path,
                        dst=te_data+'/my_face/'+f_name[-1]
                        )
            print(f'파일 이동 완료 : {f_name[-1]}')

    ## 현재 리스트가 "나의 얼굴 파일 리스트"가 아니라면 (즉, "다른 사람 얼굴 파일 리스트"라면)
    else :
        ## "다른 사람 얼굴 파일 리스트"의 파일을 Training set 폴더 안의 "다른 사람 얼굴 폴더"로 복사
        ## 이동이 잘못 되었을 경우를 생각하여 복사로 진행
        for file_path in list_tr :
            f_name = file_path.split('/')
            shutil.copy(src=file_path,
                        dst=tr_data+'/other_face/'+f_name[-1],
                        )
            print(f'파일 이동 완료 : {f_name[-1]}')

        ## "다른 사람 얼굴 파일 리스트"의 파일을 Test set 폴더 안의 "다른 사람 얼굴 폴더"로 복사
        ## 이동이 잘못 되었을 경우를 생각하여 복사로 진행
        for file_path in list_te :
            f_name = file_path.split('/')
            shutil.copy(src=file_path,
                        dst=te_data+'/other_face/'+f_name[-1]
                        )
            print(f'파일 이동 완료 : {f_name[-1]}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
파일 이동 완료 : my_face_1705.jpg
파일 이동 완료 : my_face_2261.jpg
파일 이동 완료 : my_face_1978.jpg
파일 이동 완료 : my_face_2334.jpg
파일 이동 완료 : my_face_1613.jpg
파일 이동 완료 : my_face_1034.jpg
파일 이동 완료 : my_face_1215.jpg
파일 이동 완료 : my_face_2548.jpg
파일 이동 완료 : my_face_114.jpg
파일 이동 완료 : my_face_730.jpg
파일 이동 완료 : my_face_1831.jpg
파일 이동 완료 : my_face_2451.jpg
파일 이동 완료 : my_face_1602.jpg
파일 이동 완료 : my_face_777.jpg
파일 이동 완료 : my_face_1673.jpg
파일 이동 완료 : my_face_1523.jpg
파일 이동 완료 : my_face_2630.jpg
파일 이동 완료 : my_face_66.jpg
파일 이동 완료 : my_face_532.jpg
파일 이동 완료 : my_face_1062.jpg
파일 이동 완료 : my_face_1546.jpg
파일 이동 완료 : my_face_2934.jpg
파일 이동 완료 : my_face_667.jpg
파일 이동 완료 : my_face_211.jpg
파일 이동 완료 : my_face_2681.jpg
파일 이동 완료 : my_face_1342.jpg
파일 이동 완료 : my_face_2074.jpg
파일 이동 완료 : my_face_1597.jpg
파일 이동 완료 : my_face_428.jpg
파일 이동 완료 : my_face_1151.jpg
파일 이동 완료 : my_face_2808.jpg
파일 이동 완료 : my_face_2238.jpg
파일 이동 완료 : my_face_72.jpg
파일 이동 완료 : my_face_584.jpg
파일 이동 완료 : my_face_2539.

In [18]:
## "본인 얼굴"에 대한 파일 리스트의 상위 5개 조회
img_list_my[:5]

['/content/my_face/my_face_753.jpg',
 '/content/my_face/my_face_200.jpg',
 '/content/my_face/my_face_761.jpg',
 '/content/my_face/my_face_1128.jpg',
 '/content/my_face/my_face_2092.jpg']

In [ ]:
import os

# 상위 폴더 경로 지정
base_dir = '/content/Datasets'
datasets = ['train', 'val', 'test']
classes = ['my_face', 'other_face']

# 각 폴더 내 파일 개수 확인
for dataset in datasets:
    print(f"\n--- {dataset} 데이터셋 ---")

    for class_name in classes:
        class_dir = os.path.join(base_dir, dataset, class_name)

        # 폴더가 존재하는지 확인하고 이미지 파일 개수 세기
        if os.path.exists(class_dir):
            file_count = len([f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))])
            print(f"{class_name} 폴더: {file_count}개의 이미지")
        else:
            print(f"{class_name} 폴더가 없습니다.")



--- train 데이터셋 ---
my_face 폴더: 2400개의 이미지
other_face 폴더: 2400개의 이미지

--- val 데이터셋 ---
my_face 폴더: 600개의 이미지
other_face 폴더: 600개의 이미지

--- test 데이터셋 ---
my_face 폴더: 0개의 이미지
other_face 폴더: 0개의 이미지


## 3.미션2

데이터셋의 폴더 구조를 **학습에 적합한 형태**로 만들었다면, **사전 학습된 UltraLytics YOLO-cls 모델**에 Transfer Learning을 수행합니다.

- 1) UltraLytics YOLO-cls 모델 선택
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)
- 2) 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)
- 3) 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/#inference-arguments)
- 4) 해당 UltraLytics YOLO-cls 모델을 **반드시** 저장합니다.
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.

#### (1) UltraLytics YOLO-cls 모델 선택

* **세부 요구사항**
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)

In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a model
model = YOLO(model="yolo11n-cls.yaml")  # build a new model from YAML
model=YOLO()

YOLO11x-cls summary: 309 layers, 28,458,544 parameters, 28,458,544 gradients, 111.1 GFLOPs


#### (2) UltraLytics YOLO-cls 모델 학습

* **세부 요구사항**
    - 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)

In [ ]:
model.train(model="yolo11n-cls.yaml",
            data='/content/Datasets',
            epochs=5,
            patience=5,
            verbose=True,)

Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.yaml, data=/content/Datasets, epochs=5, time=None, patience=5, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/Datasets/train... 4800 images, 0 corrupt: 100%|██████████| 4800/4800 [00:00<?, ?it/s]
val: Scanning /content/Datasets/val... 1200 images, 0 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 82 weight(decay=0.0), 83 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train34
Starting training for 5 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:07<00:00,  5.17it/s]

                   all      0.994          1


...

                   all          1          1



5 epochs completed in 0.099 hours.
Optimizer stripped from runs/classify/train34/weights/last.pt, 57.0MB
Optimizer stripped from runs/classify/train34/weights/best.pt, 57.0MB

Validating runs/classify/train34/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /content/Datasets/train... found 4800 images in 2 classes ✅ 
val: /content/Datasets/val... found 1200 images in 2 classes ✅ 
test: /content/Datasets/test... found 0 images in 0 classes: WARNING ⚠️ no images found


               classes   top1_acc   top5_acc: 100%|██████████| 38/38 [00:04<00:00,  8.72it/s]


                   all          1          1
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train34


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ace6cf070d0>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: PosixPath('runs/classify/train34')
speed: {'preprocess': 0.06998300552368164, 'inference': 1.6094785928726196, 'loss': 0.004687110582987468, 'postprocess': 0.0004221995671590169}
task: 'classify'
top1: 1.0
top5: 1.0

#### (3) UltraLytics YOLO-cls 추론

* **세부 요구사항**
    - 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/)

In [ ]:
# 모델 평가
metrics = model.val()
print(metrics.results_dict)  # 정확도, 손실 등 주요 지표 출력

WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
train: /content/Datasets/train... found 4800 images in 2 classes ✅ 
val: /content/Datasets/val... found 1200 images in 2 classes ✅ 
test: /content/Datasets/test... found 0 images in 0 classes: WARNING ⚠️ no images found


val: Scanning /content/Datasets/val... 1200 images, 0 corrupt: 100%|██████████| 1200/1200 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 75/75 [00:07<00:00,  9.65it/s]


                   all          1          1
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train33
{'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}


#### (4) UltraLytics YOLO-cls 모델 저장

* **세부 요구사항**
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.